In [62]:
import urllib.request
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 

### Ссылка на главную страницу с футбольными клубами по странам

In [64]:
r = urllib.request.urlopen('https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%BF%D0%BE_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B0%D0%BC')

In [65]:
result = r.read()

In [66]:
content = result.decode('utf-8')

In [67]:
soup = BeautifulSoup(content, 'lxml') 

In [68]:
links = []

### Парсим ее и получаем ссылки на отдельные страны

In [69]:
for div in soup.find_all('div', attrs={'class':'CategoryTreeItem'}):
    link = div.find('a').get('href')
    links.append(link)

In [70]:
len(links)

180

### Проход по каждой стране

In [73]:
teams_dict = {}
empty_team_lists = [] #список для команд стран с другой структурой страниц
empty_team_lists_links = []

for link in tqdm(links):
    page = urllib.request.urlopen('https://ru.wikipedia.org/' + link) #проходим по каждой ссылке
    result = page.read()
    content = result.decode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #находим нужный элемент в html 
        name = h1.get_text() #забираем название, напр: Футбольные клубы Англии
        if name.startswith('Категория:'):
            norm_name = name.replace('Категория:', '')
            
    teams_list = []    
    for div in soup.find_all('div', attrs={'class':'mw-category-group'}): #находим все команды в этой стране
        team_links = div.find_all('a')
        for team_link in team_links:
            team = team_link.get('title')
            #print(team)
            if team.startswith('Категория:ФК'):
                norm_team = team.replace('Категория:ФК', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное сообщество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            
            elif team.startswith('Категория:') or team.startswith('Список'):
                    continue
                
            elif team.endswith('(футбольный клуб)'):
                norm_team = team.replace('(футбольный клуб)', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное сообщество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            else:
                norm_team = team
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное сообщество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            
            teams_list.append(norm_team)
    
    teams_list1 = teams_list
    
    teams_list2 = []
    second_page_div = soup.find('div', attrs={'id':'mw-pages'})
    if second_page_div:
        a = second_page_div.find('a')
        if a.get_text() == 'Следующая страница':
            second_page = a.get('href')
            link = 'https://ru.wikipedia.org'+second_page
            page2 = urllib.request.urlopen(link)
            result2 = page2.read()
            content2 = result2.decode('utf-8')
            soup2 = BeautifulSoup(content2, 'lxml')
            h2 = soup2.find('div', attrs={'id':'mw-pages'})
            for a in h2.find_all('a'):
                team2 = a.get('title')
                #print(team2)
                if team2.startswith('Категория:ФК'):
                    norm_team2 = team2.replace('Категория:ФК', '')
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(футбольный клуб.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(спортивное общество\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                elif team2.startswith('Категория:') or team2.startswith('Список'):
                    continue    
                elif team.endswith('(футбольный клуб)'):
                    norm_team2 = team2.replace('(футбольный клуб)', '')
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(футбольный клуб.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(спортивное общество\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                else:
                    norm_team2 = team2
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(футбольный клуб.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(спортивное общество\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб\)','', norm_team2)
                    norm_team2 = re.sub(r'\(женский футбольный клуб.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                #print(norm_team2)
                
                teams_list2.append(norm_team2)
            #print(teams_list2)
    
    if teams_list2:
        full_teams_list = set(teams_list1 + teams_list2)
    else:
        full_teams_list = set(teams_list1)
                

    if full_teams_list: #если список с командами не пустой - добавляем в словарь
        teams_dict[norm_name] = full_teams_list
    else:
        empty_team_lists.append(norm_name) #если список команд пустой - добавляем в список с такими же странами
        empty_team_lists_links.append(link) 

100%|████████████████████████████████████████████████████████████████████████████████| 180/180 [01:34<00:00,  1.90it/s]


### Пример из полученного словаря

Достались не все команды, так как в википедии есть страницы с элементами "следующая страница". Нужно доделать парсинг таких страниц до конца, чтобы получить все команды. 

In [74]:
teams_dict['Футбольные клубы России']

{'АПК',
 'Авангард',
 'Авангард Камышин',
 'Авангард Курск',
 'Автозапчасть',
 'Азамат',
 'Академия',
 'Академия футбола имени В. Понедельника',
 'Акрон',
 'Алания',
 'Алания-Д',
 'Амкар',
 'Амур Благовещенск',
 'Амур-2010',
 'Амурский уголь',
 'Ангара',
 'Ангушт',
 'Анжи',
 'Анжи-2',
 'Анжи-Юниор',
 'Апатит',
 'Арарат',
 'Арзамас',
 'Армавир',
 'Арсенал',
 'Арсенал Тула',
 'Арсенал-2',
 'Асмарал-д',
 'Астрахань',
 'Атом',
 'Афипс',
 'Ахмат',
 'БСК',
 'Байкал',
 'Балаково',
 'Балтика',
 'Балтика-2',
 'Балтика-Тарко',
 'Батайск-2007',
 'Башсельмаш',
 'Бежица',
 'Белогорск',
 'Березники',
 'Биолог-Новокубанск',
 'Биробиджан',
 'Благовещенск',
 'Бриз',
 'Бугульма-Рунако',
 'Булат',
 'Бурятия',
 'Вайнах',
 'Велес',
 'Венец',
 'Вест',
 'Видное',
 'Виктория',
 'Витязь',
 'Владикавказ',
 'Волга',
 'Волга Нижний Новгород',
 'Волгарь',
 'Волгарь-Астрахань',
 'Волгоград',
 'Волгодонск',
 'Волгомост',
 'Волжанин',
 'Волжанка',
 'Вологда',
 'Вологжанка',
 'Волочанин',
 'Волхов',
 'Волхов (футзальн

### Футбольные клубы, которые не запарсились из-за другой структуры веб-страницы

Нужно просто добавить еще один цикл для данных стран

In [75]:
empty_team_lists[:10]

['Футбольные клубы по непризнанным и частично признанным государствам',
 'Женские футбольные клубы по странам',
 'Футбольные клубы Американского Самоа',
 'Футбольные клубы Бангладеш',
 'Футбольные клубы Бахрейна',
 'Футбольные клубы Белиза',
 'Футбольные клубы Бенина',
 'Футбольные клубы Бразилии',
 'Футбольные клубы Бурунди',
 'Футбольные клубы Бутана']

Ссылки на страны к которым нужно применить доп цикл

In [76]:
empty_team_lists_links[:3]

['/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%BF%D0%BE_%D0%BD%D0%B5%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC_%D0%B8_%D1%87%D0%B0%D1%81%D1%82%D0%B8%D1%87%D0%BD%D0%BE_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC_%D0%B3%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2%D0%B0%D0%BC',
 '/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%96%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B5_%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%BF%D0%BE_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B0%D0%BC',
 '/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%90%D0%BC%D0%B5%D1%80%D0%B8%D0%BA%D0%B0%D0%BD%D1%81%D0%BA%D0%BE%D0%B3%D0%BE_%D0%A1%D0%B0%D0%BC%D0%BE%D0%B0']

### Добавление в словарь оставшихся клубов 

In [77]:
bad_links = []

for link in tqdm(empty_team_lists_links):
    #print('https://ru.wikipedia.org' + link)
    page = urllib.request.urlopen('https://ru.wikipedia.org' + link) 
    result = page.read()
    content = result.decode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #находим нужный элемент в html 
        name = h1.get_text() #забираем название, напр: Футбольные клубы Англии
        if name.startswith('Категория:'):
            norm_name = name.replace('Категория:', '')
    
    teams_list = []    
    for div in soup.find_all('div', attrs={'class':'mw-category-generated'}): #находим все команды в этой стране
        #print(div)
        team_links = div.find_all('a')
        #print(team_links)
        for team_link in team_links:
            team = team_link.get('title')
            if team.startswith('Категория:ФК'):
                norm_team = team.replace('Категория:ФК', '') 
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                
            elif team.endswith('(футбольный клуб)'):
                norm_team = team.replace('(футбольный клуб)', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                
            elif team.startswith('Категория:'):
                bad_links.append(team_link.get('href'))
                #norm_team = norm_team.strip(' ')
                
            else:
                norm_team = team
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
                norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
                norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                    
            teams_list.append(norm_team)
    teams_list = set(teams_list)
    teams_dict[norm_name] = teams_list
            
    

100%|██████████████████████████████████████████████████████████████████████████████████| 76/76 [00:37<00:00,  2.01it/s]


### Оставшиеся плохие ссылки

In [78]:
for link in bad_links[:3]:
    print('https://ru.wikipedia.org' + link)

https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%90%D0%B1%D1%85%D0%B0%D0%B7%D0%B8%D0%B8
https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%9A%D0%BE%D1%81%D0%BE%D0%B2%D0%B0
https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%9D%D0%B0%D0%B3%D0%BE%D1%80%D0%BD%D0%BE-%D0%9A%D0%B0%D1%80%D0%B0%D0%B1%D0%B0%D1%85%D1%81%D0%BA%D0%BE%D0%B9_%D0%A0%D0%B5%D1%81%D0%BF%D1%83%D0%B1%D0%BB%D0%B8%D0%BA%D0%B8


### Парсинг клубов отдельной страны/города по ссылке 

In [83]:
link = 'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A4%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D0%BA%D0%BB%D1%83%D0%B1%D1%8B_%D0%9A%D0%B8%D0%B5%D0%B2%D0%B0'
page = urllib.request.urlopen(link) 
result = page.read()
content = result.decode('utf-8')
soup = BeautifulSoup(content, 'lxml')
for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #находим нужный элемент в html 
        name = h1.get_text() #забираем название, напр: Футбольные клубы Англии
        if name.startswith('Категория:'):
            norm_name = name.replace('Категория:', '')
            
teams_list = []    
for div in soup.find_all('div', attrs={'class':'mw-category-generated'}): #находим все команды в этой стране
    #print(div)
    team_links = div.find_all('a')
    #print(team_links)
    for team_link in team_links:
        team = team_link.get('title')
        if team.startswith('Категория:ФК'):
            norm_team = team.replace('Категория:ФК', '') 
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
            norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                
        elif team.endswith('(футбольный клуб)'):
            norm_team = team.replace('(футбольный клуб)', '')
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
            norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                
        elif team.startswith('Категория:'):
            bad_links.append(team_link.get('href'))
            #norm_team = norm_team.strip(' ')
                
        else:
            norm_team = team
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(футбольный клуб.+\)','', norm_team)
            norm_team = re.sub(r'\(футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(спортивное общество\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                    
        teams_list.append(norm_team)
teams_list = set(teams_list)
teams_dict[norm_name] = teams_list


In [84]:
teams_dict['Футбольные клубы Киева']

{'Арсенал-Киев',
 'Атекс-СДЮШОР-16',
 'Динамо Киев',
 'Локомотив',
 'Оболонь',
 'Оболонь-Бровар',
 'Спартак',
 'Темп',
 'ЦСКА Киев'}

### Весь словарь 

In [85]:
teams_dict

{'Футбольные клубы Австралии': {'Аделаида Юнайтед',
  'Брисбен Роар',
  'Брунсвик Ювентус',
  'Вестерн Сидней Уондерерс',
  'Голд-Кост Юнайтед',
  'Квинсленд Лайонс',
  'Мельбурн Виктори',
  'Мельбурн Сити',
  'Ньюкасл Юнайтед Джетс',
  'Перт Глори',
  'Сентрал Кост Маринерс',
  'Сидней',
  'Сидней Юнайтед'},
 'Футбольные клубы Австрии': {'Адмира Ваккер Мёдлинг',
  'Альтах',
  'Аустрия',
  'Аустрия Вена',
  'Аустрия Кернтен',
  'Брегенц',
  'Ваккер',
  'Виенна Крикет',
  'Винер Атлетикшпорт-Клуб',
  'Винер Шпорт-Клуб',
  'Винер-Нойштадт',
  'Вольфсберг',
  'ГАК',
  'Граткорн',
  'Грёдиг',
  'Капфенберг',
  'ЛАСК',
  'Леобен',
  'Линц',
  'Лиферинг',
  'Лустенау 07',
  'Маттерсбург',
  'Мауэрверк',
  'Парндорф',
  'Рапид',
  'Рапид Вена',
  'Ред Булл',
  'Ред Булл Зальцбург',
  'Рид',
  'Санкт-Пёльтен',
  'Сваровски-Тироль',
  'Суперфунд',
  'Тироль',
  'Флоридсдорф',
  'Форвертс Штайр',
  'Фёклабрук',
  'Фёрст',
  'Хакоах',
  'Хартберг',
  'Хорн',
  'Штурм',
  'Юниорс'},
 'Футбольные к